# Imports

In [1]:
import socket
import numpy as np
import datetime

# TCP Tests

In [3]:
IP_ADDRESS = "192.168.0.17"
TCP_Port = 23
UDP_Port = 4660

In [5]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM, 0)
sock.settimeout(5)
sock.connect((IP_ADDRESS, TCP_Port))

timeout: timed out

In [69]:
sendHandshake = bytearray([0xa3,0x00,0x00,0x00]) + 0xffff.to_bytes(4,'big')
print(sendHandshake)

bytearray(b'\xa3\x00\x00\x00\x00\x00\xff\xff')


In [79]:
with socket.socket(socket.AF_INET,socket.SOCK_STREAM,0) as sock:
    sock.settimeout(5)
    sock.connect((IP_ADDRESS,TCP_Port))
    sock.send(sendHandshake)
    recData = sock.recv(1024)    
    print(recData)
    sock.close()

b'\x00\x00\x00\x00'


In [86]:
with socket.socket(socket.AF_INET,socket.SOCK_STREAM,0) as sock:
    sock.settimeout(5)
    sock.connect((IP_ADDRESS,TCP_Port))
    # sock.send(sendHandshake)
    recData = sock.recv(1024)    
    print(len(recData),recData)
    sock.close()

23 b'Z\x03#s\x0f\x15\x00\xbbZ\x0f\x0b\xef\x0f\x01\xa0)Z\x0f\xcb\xf0\x0f%\xd1'


# Test input data

In [79]:
def instrument_time(input=None,mode='seconds'):
    '''
    Function to convert time from bytes to seconds and the reverse.
    Time is defined as seconds since 2008 and stored in 5 bytes.
    4 bytes for seconds and 1 byte for subsections

    Parameters
    ---------
    input : Time input as 5 bytes, seconds, or datetime, optional
        If input is None, the current time will be output as 5 bytes
    mode : str
        'seconds' : output time in seconds
        'datetime' : output time in datetime

    '''
    if input is None:
        secondsSince2008 = (datetime.datetime.now() - datetime.datetime(2008,1,1,0,0)).total_seconds()
        secondsBytes = int(secondsSince2008).to_bytes(4,'big')
        subsecondsBytes = int(secondsSince2008 % 1 * 2**8).to_bytes(1,'big')
        return secondsBytes + subsecondsBytes
    elif isinstance(input,(bytes,bytearray)):
        intSeconds = int.from_bytes(input[:4],'big')
        intSubSeconds = input[5]
        secondsSince2008 = intSeconds + (intSubSeconds / 2**8)
        if mode == 'seconds':
            return secondsSince2008
        elif mode == 'datetime':
            return datetime.datetime(2008,1,1) + datetime.timedelta(seconds=secondsSince2008)
    elif isinstance(input,(int,float)):
        return datetime.datetime(2008,1,1,0,0) + datetime.timedelta(seconds=input)
    elif isinstance(input,datetime):
        return (input - datetime.datetime(2008,1,1,0,0)).total_seconds()

In [6]:
with open('Test TCP Data/testData_1024.dat') as openFile:
    testTCPData = bytes.fromhex(openFile.read().replace(':',''))

In [14]:
for i in range(len(testTCPData)//8):
    if hex(testTCPData[8*i]) == '0x6c':
        print(i)

7
29
42
59
74
86
104
121


In [33]:
timeBytes = testTCPData[8*7:8*8]
print(timeBytes.hex(':'))

6c:1d:49:a2:96:1c:4c:8e


In [38]:
timeBytes[1:5].hex(':')

'1d:49:a2:96'

In [61]:
timeBytes[5:10].hex(':')

'1c:4c:8e'

In [86]:
instrument_time(timeBytes[1:7],mode='datetime').timestamp()

1690525014.296875

In [93]:
(instrument_time(timeBytes[1:7],mode='datetime') - datetime.datetime(1970,1,1)).total_seconds()

1690510614.296875

In [51]:
len(timeBytes[1:6])

5

In [67]:
bytes([0x23,0x43,0xb3])[100]

IndexError: index out of range

In [27]:
hist1 = np.column_stack((np.arange(1024)/1024,np.zeros(1024)))

In [31]:
hist1[23,1]

0.0

In [45]:
np.arange(1024)/1023 == np.linspace(0,1,1024)

array([ True,  True,  True, ...,  True,  True,  True])

In [17]:
bins = np.linspace(0,1,1024)

In [21]:
bins[1023]

1.0

In [65]:
4*np.random.rand()

1.4598691551728198

In [70]:
x = np.arange(0,10,1)
y = x * 2
data = np.column_stack((x,y))
data

array([[ 0,  0],
       [ 1,  2],
       [ 2,  4],
       [ 3,  6],
       [ 4,  8],
       [ 5, 10],
       [ 6, 12],
       [ 7, 14],
       [ 8, 16],
       [ 9, 18]])

In [71]:
data[:,0]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [72]:
data[:,1]

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])